#Step 4: tuning dei modelli e ricerca dei migliori iperparametri attraverso GridSearch#

In [ ]:
from datetime import datetime, timedelta, date
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_parquet("/content/drive/MyDrive/Colab Notebooks/Models4.parquet")

In [ ]:
Regressors=['n_restituzioni','Dummy_neg','Recency','Frequency','Monetary','T']
var_num=['n_restituzioni','Recency','Frequency','Monetary','T']
X=df.drop('purchaseamount_m4', axis=1)
y=df['purchaseamount_m4']
#Splitto in train e test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25, random_state=29)

**Standardizzazione delle variabili quantitative**

In [ ]:
from sklearn.preprocessing import StandardScaler
std_scl= StandardScaler()
X_train[var_num] = std_scl.fit_transform(X_train[var_num])
X_test[var_num] =std_scl.transform(X_test[var_num])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-do

In [ ]:
X_train,X_test=X_train[var_num],X_test[var_num]

In [ ]:
X_train.head()

,n_restituzioni,Recency,Frequency,Monetary,T
30378,0.267588,0.347551,-0.135540,-0.246653,0.242920
26401,1.228812,-0.063033,0.917423,0.756197,0.242920
10531,2.510445,-0.268326,-0.517605,-0.701132,0.148228
26533,-0.533432,0.244905,-1.075163,-1.418428,-0.088501
7135,-0.132922,-0.576264,0.052750,0.248421,0.337612


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import numpy as np

#Lineare con T senza dummy
lin_reg= LinearRegression()
lin_reg.fit(X_train, y_train)
y_pred=lin_reg.predict(X_train)
y_pred_test=lin_reg.predict(X_test)
#Residui
y_e_train=y_train - y_pred
y_e_test=y_test - y_pred_test



In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'n_estimators': [300, 900], 'max_depth': [2,8],'min_samples_split':[0.02,0.2]}
]

forest_reg = RandomForestRegressor()

grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                          scoring= 'neg_mean_squared_error',
                          return_train_score= True)

grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='deprecated', n_jo

In [ ]:
grid_search.best_params_

{'max_depth': 8, 'min_samples_split': 0.02, 'n_estimators': 900}

In [ ]:
grid_search.best_estimator_

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=8, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=0.02, min_weight_fraction_leaf=0.0,
                      n_estimators=900, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [ ]:
cvres= grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

1348.4395579461655 {'max_depth': 2, 'min_samples_split': 0.02, 'n_estimators': 300}
1342.7732566490179 {'max_depth': 2, 'min_samples_split': 0.02, 'n_estimators': 900}
1394.8410526273283 {'max_depth': 2, 'min_samples_split': 0.2, 'n_estimators': 300}
1393.7276784264702 {'max_depth': 2, 'min_samples_split': 0.2, 'n_estimators': 900}
1210.4941489011724 {'max_depth': 8, 'min_samples_split': 0.02, 'n_estimators': 300}
1208.9767821651608 {'max_depth': 8, 'min_samples_split': 0.02, 'n_estimators': 900}
1364.241668140122 {'max_depth': 8, 'min_samples_split': 0.2, 'n_estimators': 300}
1370.561230945698 {'max_depth': 8, 'min_samples_split': 0.2, 'n_estimators': 900}


In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'n_estimators': [700,900,1100], 'max_depth': [6,8,10],'min_samples_split':[0.01,0.02,0.05]}
]

forest_reg = RandomForestRegressor()

grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                          scoring= 'neg_mean_squared_error',
                          return_train_score= True)

grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='deprecated', n_jo

In [ ]:
grid_search.best_params_

{'max_depth': 8, 'min_samples_split': 0.01, 'n_estimators': 1100}

In [ ]:
grid_search.best_estimator_

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=8, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=0.01, min_weight_fraction_leaf=0.0,
                      n_estimators=1100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [ ]:
cvres= grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

1202.7118205365807 {'max_depth': 6, 'min_samples_split': 0.01, 'n_estimators': 700}
1203.5540511247916 {'max_depth': 6, 'min_samples_split': 0.01, 'n_estimators': 900}
1199.8680958675711 {'max_depth': 6, 'min_samples_split': 0.01, 'n_estimators': 1100}
1210.537232522872 {'max_depth': 6, 'min_samples_split': 0.02, 'n_estimators': 700}
1202.9221984339702 {'max_depth': 6, 'min_samples_split': 0.02, 'n_estimators': 900}
1209.374860978885 {'max_depth': 6, 'min_samples_split': 0.02, 'n_estimators': 1100}
1283.0700019685553 {'max_depth': 6, 'min_samples_split': 0.05, 'n_estimators': 700}
1288.8147511580216 {'max_depth': 6, 'min_samples_split': 0.05, 'n_estimators': 900}
1282.9843571147173 {'max_depth': 6, 'min_samples_split': 0.05, 'n_estimators': 1100}
1202.9638785983823 {'max_depth': 8, 'min_samples_split': 0.01, 'n_estimators': 700}
1205.2494424245504 {'max_depth': 8, 'min_samples_split': 0.01, 'n_estimators': 900}
1198.5878266178831 {'max_depth': 8, 'min_samples_split': 0.01, 'n_estimator

In [ ]:
feature_importances = grid_search.best_estimator_.feature_importances_
feature_importances

array([0.23500703, 0.00102575, 0.27724563, 0.4759765 , 0.01074509])

In [ ]:
#RANDOM FOREST
from sklearn.ensemble import RandomForestRegressor

y_trans= np.log1p(y_train)
rnd_forest= RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=10, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=0.01, min_weight_fraction_leaf=0.0,
                      n_estimators=900, n_jobs=-1, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)
rnd_forest.fit(X_train,y_train)
y_pred=rnd_forest.predict(X_train)
y_pred_test=rnd_forest.predict(X_test)

In [ ]:
#EQM train
for_mse = mean_squared_error(y_train, y_pred)
for_rmse = np.sqrt(for_mse)
for_rmse

559.2106976104324

In [ ]:
#R2 train
from sklearn.metrics import r2_score
r2=r2_score(y_train,  y_pred)
r2

0.864153529782089

In [ ]:
#EQM test
y_pred_test= rnd_clf.predict(X_test)
for_mse = mean_squared_error(y_test,y_pred_test)
for_rmse = np.sqrt(for_mse)
for_rmse

395.60569947700384

In [ ]:
#R2 test
r2=r2_score(y_test, y_pred_test)
r2

0.792804434172072

In [ ]:
#CROSS VALIDATION
from sklearn.model_selection import cross_val_score
scores= cross_val_score(rnd_clf, X_train, y_train,
                       scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-scores)
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard Dev:", scores.std())

In [ ]:
display_scores(forest_rmse_scores)

Scores: [ 405.33658339 3191.83407474  500.88056377  388.66064328  389.98455281
  439.03771586  677.65673185  418.16145558  400.84653121  402.41228477]
Mean: 721.4811137252789
Standard Dev: 827.660816502421


In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'n_estimators': [300], 'max_depth': [10],'min_samples_split':[0.01], 'max_leaf_nodes':[2,3,7,8]}
]

forest_reg = RandomForestRegressor()

grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                          scoring= 'neg_mean_squared_error',
                          return_train_score= True)

grid_search.fit(X_train, y_e_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='deprecated', n_jo

In [ ]:
grid_search.best_params_

{'max_depth': 10,
 'max_leaf_nodes': 8,
 'min_samples_split': 0.01,
 'n_estimators': 300}

In [ ]:
grid_search.best_estimator_

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=10, max_features='auto', max_leaf_nodes=8,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=0.01, min_weight_fraction_leaf=0.0,
                      n_estimators=300, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [ ]:
#RANDOM FOREST
rnd_forest= RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=10, max_features='auto', max_leaf_nodes=8,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=0.01, min_weight_fraction_leaf=0.0,
                      n_estimators=300, n_jobs=None, oob_score=True,
                      random_state=None, verbose=0, warm_start=False)
rnd_forest.fit(X_train,y_train)
y_pred_e=rnd_forest.predict(X_train)
y_pred_test_e=rnd_forest.predict(X_test)

In [ ]:
from sklearn.model_selection import GridSearchCV
y_hat=y_pred+y_pred_e
y_hat_test=y_pred_test+y_pred_test_e
param_grid = [
    {'n_estimators': [300, 900], 'max_depth': [2,8],'min_samples_split':[0.02,0.2]}
]

forest_reg = RandomForestRegressor()

grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                          scoring= 'neg_mean_squared_error',
                          return_train_score= True)

grid_search.fit(X_train, y_hat)

GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='deprecated', n_jo

In [ ]:
grid_search.best_params_

{'max_depth': 8, 'min_samples_split': 0.02, 'n_estimators': 300}

In [ ]:
grid_search.best_estimator_

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=8, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=0.02, min_weight_fraction_leaf=0.0,
                      n_estimators=300, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [ ]:
cvres= grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

1910.3244580428873 {'max_depth': 2, 'min_samples_split': 0.02, 'n_estimators': 300}
1901.893876465289 {'max_depth': 2, 'min_samples_split': 0.02, 'n_estimators': 900}
1994.4757679288316 {'max_depth': 2, 'min_samples_split': 0.2, 'n_estimators': 300}
1993.7692941105206 {'max_depth': 2, 'min_samples_split': 0.2, 'n_estimators': 900}
1598.7444612913991 {'max_depth': 8, 'min_samples_split': 0.02, 'n_estimators': 300}
1605.8994693330553 {'max_depth': 8, 'min_samples_split': 0.02, 'n_estimators': 900}
1947.4310691950009 {'max_depth': 8, 'min_samples_split': 0.2, 'n_estimators': 300}
1948.8540174048544 {'max_depth': 8, 'min_samples_split': 0.2, 'n_estimators': 900}


In [ ]:
from sklearn.model_selection import GridSearchCV
y_hat=y_pred+y_pred_e
y_hat_test=y_pred_test+y_pred_test_e
param_grid = [
    {'n_estimators': [200,300, 400], 'max_depth': [6,8,10],'min_samples_split':[0.01,0.02,0.05]}
]

forest_reg = RandomForestRegressor()

grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                          scoring= 'neg_mean_squared_error',
                          return_train_score= True)

grid_search.fit(X_train, y_hat)

GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='deprecated', n_jo

In [ ]:
grid_search.best_params_

{'max_depth': 10, 'min_samples_split': 0.01, 'n_estimators': 300}

In [ ]:
grid_search.best_estimator_

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=10, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=0.01, min_weight_fraction_leaf=0.0,
                      n_estimators=300, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [ ]:
cvres= grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

1603.770358875213 {'max_depth': 6, 'min_samples_split': 0.01, 'n_estimators': 200}
1586.0580741797787 {'max_depth': 6, 'min_samples_split': 0.01, 'n_estimators': 300}
1597.4279588989273 {'max_depth': 6, 'min_samples_split': 0.01, 'n_estimators': 400}
1605.073623331674 {'max_depth': 6, 'min_samples_split': 0.02, 'n_estimators': 200}
1595.6031290900762 {'max_depth': 6, 'min_samples_split': 0.02, 'n_estimators': 300}
1613.4953868503326 {'max_depth': 6, 'min_samples_split': 0.02, 'n_estimators': 400}
1780.110976649898 {'max_depth': 6, 'min_samples_split': 0.05, 'n_estimators': 200}
1764.822750699152 {'max_depth': 6, 'min_samples_split': 0.05, 'n_estimators': 300}
1771.787275423462 {'max_depth': 6, 'min_samples_split': 0.05, 'n_estimators': 400}
1582.9865260462102 {'max_depth': 8, 'min_samples_split': 0.01, 'n_estimators': 200}
1596.705159903886 {'max_depth': 8, 'min_samples_split': 0.01, 'n_estimators': 300}
1598.9342868746603 {'max_depth': 8, 'min_samples_split': 0.01, 'n_estimators': 400

In [ ]:
from scikit-learn.metrics import mean_absolute_percentage_error

SyntaxError: ignored

In [ ]:
df.head()

,id,n_restituzioni,Dummy_neg,Recency,Frequency,Monetary,T,purchaseamount_m4
0,4330474628,0,0,7,8,21.84,7,1088.07
1,4330475834,0,0,7,16,47.21,7,206.63
2,4330479043,0,0,6,31,111.72,6,1484.16
3,4330479752,0,0,6,3,29.96,6,365.46
4,4334046217,0,0,0,14,29.17,5,904.44


In [ ]:
R2_score={}
for x in range(2,20):
  rnd_forest= RandomForestRegressor(criterion='mse',max_depth=x,n_estimators=y, n_jobs=-1)
  #fit
  #predict
  #R2
  R2_score[x] = dict()
  R2_score[x] = {"r2_train": r2_train, "r2_test": r2_test}
